<a href="https://colab.research.google.com/github/Antonsen2/wildfire-ai/blob/research_2.0/optimization/notebooks/optimization_with_optuna_object_detection_version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install albumentations==0.4.6
!pip install pycocotools
!pip install torchvision
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj')

In [ ]:
%%shell

git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

fatal: destination path 'vision' already exists and is not an empty directory.
HEAD is now at 2f40a483d7 [v0.8.X] .circleci: Add Python 3.9 to CI (#3063)


In [ ]:
import os
import time
import copy
import optuna
import torchvision
import torch
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as transform
import albumentations as A
import torch.nn as nn

from torchvision.models.detection.rpn import AnchorGenerator
from torchvision import datasets, models, transforms
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.transforms as transforms
from torchvision.models.detection import FasterRCNN
from engine import train_one_epoch, evaluate
from torch.utils.data import Dataset, DataLoader
from typing import Any
from PIL import Image

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/train_annotations.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/valid_annotations.csv')

train_images = '/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/train'
valid_images = '/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/valid'

train_df.tail()

,filename,width,height,class,xmin,ymin,xmax,ymax
1200,fire-9.jpg,640,480,fire,65,306,208,355
1201,fire-9.jpg,640,480,fire,1,350,89,377
1202,fire-9.jpg,640,480,fire,496,363,527,386
1203,fire-9.jpg,640,480,fire,459,342,486,371
1204,fire-9.jpg,640,480,fire,612,332,640,353


In [ ]:
GPU = True
DEVICE = torch.device('cuda') if GPU else torch.device('cpu')

LABELS = train_df['class'].unique()
NUM_OF_CLASSES = len(LABELS)+1
IN_FEATURES = model.roi_heads.box_predictor.cls_score.in_features

SAVE_PATH = '/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/models/'
MODEL_NAME = 'model_include_fire.pt'

In [ ]:
def collate_fn(batch: tuple) -> tuple:
  return tuple(zip(*batch))

def train(model, train_loader, valid_loader, epochs) -> None:
  params = [param for param in model.parameters() if param.requires_grad]
  optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)  # TODO: Optimize params

  lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)  # TODO: Optimize params

  for epoch in range(epochs):
    train_one_epoch(
        model=model,
        optimizer=optimizer,
        data_loader=train_loader,
        device=DEVICE,
        epoch=epoch,
        print_freq=len(train_loader)
    )
    
    lr_schedule.step()
    evaluate(
        model=model,
        data_loader=valid_loader,
        device=DEVICE
    )

In [ ]:
def plot_images(images, targets) -> None:
  max_images = 4
  img_counter = 0

  for image, target in zip(images, targets):
    if img_counter == max_images:
      break

    img_counter += 1

    sample = image.permute(1,2,0).cpu().numpy()
    boxes = target['boxes'].cpu().numpy().astype(np.int32)
    labels = target['labels'].cpu().numpy()

    fig, ax = plt.subplots(1, 1, figsize=(10,8))

    for i, box in enumerate(boxes):
      cv2.rectangle(
          img=sample,
          pt1=(box[0], box[1]),
          pt2=(box[2], box[3]),
          color=(1,0,0),
          thickness=1
      )
      cv2.putText(
          img=sample,
          text=labels[i],
          org=(box[0], box[1]-10),
          fontFace=cv2.FONT_HERSHEY_DUPLEX,
          fontScale=0.3,
          color=(0,0,0),
          thickness=1
      )
    
    ax.set_axis_off();
    ax.imshow((sample * 255).astype(np.uint8))

In [ ]:
class LabelMap:
  def __init__(self, labels: list) -> None:
    self._map = {c: i+1 for i, c in enumerate(labels)}
    self.reversed_map = {i: c for i, c in enumerate(labels)}

  def fit(self, df: pd.DataFrame, col: str) -> pd.DataFrame:
    df[col] = df[col].map(self._map)
    return df

In [ ]:
class WildfireDataset(Dataset):
  def __init__(self, df: pd.DataFrame, img_path: str, labels: list, transforms: Any = None, **kwargs) -> None:
    super().__init__(**kwargs)
    self.df = df
    self.img_path = img_path
    self.labels = labels
    self.images = self.df['filename'].unique()
    self.transforms = transforms

  def __len__(self) -> int:
    return len(self.images)

  def __getitem__(self, i: int) -> tuple:
    img_file = os.path.join(self.img_path, self.images[i])

    img = cv2.imread(img_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    img = img/255.0

    img_data = self.df.loc[self.df['filename'] == self.images[i]]

    xmins = img_data['xmin'].values
    ymins = img_data['ymin'].values
    xmaxs = img_data['xmax'].values
    ymaxs = img_data['ymax'].values

    boxes = torch.as_tensor(np.stack([xmins, ymins, xmaxs, ymaxs], axis=1), dtype=torch.float32)
    labels = torch.as_tensor(img_data['class'].values, dtype=torch.int64)
    _id = torch.tensor([i])

    areas = (boxes[:,3] - boxes[:,1]) * (boxes[:,2] - boxes[:,0])
    areas = torch.as_tensor(areas, dtype=torch.float32)

    iscrowd = torch.zeros((len(labels),), dtype=torch.int64)

    target = dict()
    target['boxes'] = boxes
    target['labels'] = labels
    target['image_id'] = _id
    target['area'] = areas
    target['iscrowd'] = iscrowd

    if self.transforms:
      transformed = self.transforms(image=img, bboxes=boxes, labels=labels)
      img = transformed['image']
      target['boxes'] = torch.as_tensor(transformed['bboxes'], dtype=torch.float32)
    
    return torch.as_tensor(img, dtype=torch.float32), target

  def get_h_w(self, image: str) -> tuple:
    """Get height and width of image"""
    img_data = self.df.loc[self.df['filename'] == image]
    return img_data['width'].values[0], img_data['height'].values[0]

In [ ]:
label_map = LabelMap(LABELS)  # 1='smoke', 2='fire'

train_df = label_map.fit(train_df, 'class')
valid_df = label_map.fit(valid_df, 'class')

train_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,ck0qd8gs6ko7j0721x25cv4o3_jpeg.rf.005f5707706e...,640,480,1,125,190,177,286
1,ck0t40rhdz68s0a46ekx049a6_jpeg.rf.00403179fe5f...,640,480,1,326,207,494,249
2,ck0m0ch9ugnna07940o8x989j_jpeg.rf.0101cdb46a16...,640,480,1,308,166,582,257
3,ck0rr6bfa9b3w0721aw5unwdy_jpeg.rf.00982c053d66...,640,480,1,241,204,310,244
4,ck0uk75x5ysls0721e5a9j891_jpeg.rf.00d7fd8503e1...,640,480,1,523,208,619,288


In [ ]:
# TODO: Optimize params
train_transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomAffine(degrees=15, 
                translate=(0.1, 0.1),
                scale=(0.8, 1.2)),
                transforms.ToTensor()
                ])
valid_transform = transforms.Compose([transforms.ToTensor()])

data_transforms = {"train": train_transform, "valid": valid_transform}

In [ ]:
train_dataset = WildfireDataset(train_df, train_images, LABELS, train_transform)
valid_dataset = WildfireDataset(valid_df, valid_images, LABELS, valid_transform)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn
)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

dataloaders = {
    "train": train_dataloader,
    "valid": valid_dataloader
}

In [ ]:
data_dir = {'train': train_images, 'valid': valid_images}

# Load the datasets using the ImageFolder class
image_datasets = {x: datasets.ImageFolder(data_dir[x], data_transforms[x])
                  for x in ['train', 'valid']}

# Load the data into PyTorch dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'valid']}

# Get the dataset sizes
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}

# Get the class names
class_names = image_datasets['train'].classes

# Get the device (CPU or GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# images, targets = next(iter(train_dataloader))
# plot_images(images, targets)

In [ ]:
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(IN_FEATURES, NUM_OF_CLASSES)
model = model.to(DEVICE) 

RuntimeError: ignored

In [ ]:
train(model, train_dataloader, valid_dataloader, epochs=20)

In [ ]:
torch.save(model.state_dict(), SAVE_PATH + MODEL_NAME)

# Optina optimization

In [ ]:
def get_model(model_name: str = "resnet18"):  
    if model_name == "resnet18":
        model = models.resnet18(pretrained=True)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, 2)
    elif model_name == "alexnet":
        model = models.alexnet(pretrained=True)
        in_features = model.classifier[1].in_features
        model.classifier = nn.Linear(in_features, 2)
    elif model_name == "vgg16":
        model = models.vgg16(pretrained=True)
        in_features = model.classifier[0].in_features
        model.classifier = nn.Linear(in_features, 2)
    elif model_name == "fasterrcnn_resnet50_fpn":
        model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    elif model_name == "retinanet_resnet50_fpn":
        model = models.detection.retinanet_resnet50_fpn(pretrained=True)
    elif model_name == "maskrcnn_resnet50_fpn":
        model = models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    elif model_name == "keypointrcnn_resnet50_fpn":
        model = models.detection.keypointrcnn_resnet50_fpn(pretrained=True)
    return model

In [ ]:
## New one. 

def train_model(trial, model, criterion, optimizer, num_epochs=25, gpu=True):
    device = torch.device('cuda') if gpu else torch.device('cpu')
    
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() 
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()
        
        trial.report(epoch_acc, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model, best_acc

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

def create_data_loaders(train_df, valid_df, train_images, valid_images, batch_size, num_workers, shuffle, train_transform, val_transform, test_transform):
    # Define a custom dataset for the training data
    class WildfireDataset(torch.utils.data.Dataset):
        def __init__(self, df, images_dir, transform=None):
            self.df = df
            self.images_dir = images_dir
            self.transform = transform
        
        def __len__(self):
            return len(self.df)
        
        def __getitem__(self, idx):
            image_path = os.path.join(self.images_dir, self.df.iloc[idx, 0])
            image = Image.open(image_path).convert("RGB")
            
            if self.transform:
                image = self.transform(image)
            
            return image, self.df.iloc[idx, 1]
    
    # Define the transformation to be applied to the images
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # Create the custom datasets
    train_dataset = WildfireDataset(train_df, train_images, transform)
    val_dataset = WildfireDataset(valid_df, valid_images, transform)
    test_dataset = WildfireDataset(valid_df, valid_images, transform)
    
    # Create the data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle,
                              num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle,
                            num_workers=num_workers)
    test_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle,
                            num_workers=num_workers)
    
    return train_loader, val_loader, test_loader

In [ ]:
def objective(trial):
    
    # Hyperparameters we want to optimize
    params = {
        "model_name": trial.suggest_categorical('model_name',["resnet18", "alexnet", "vgg16", "fasterrcnn_resnet50_fpn", "retinanet_resnet50_fpn", "maskrcnn_resnet50_fpn", "keypointrcnn_resnet50_fpn"]),
        "lr": trial.suggest_float('lr', 1e-4, 1e-2),
        "optimizer_name": trial.suggest_categorical('optimizer_name',["SGD", "Adam", "Adadelta", "Adagrad", "RMSprop", "ASGD ", "LBFGS", "AdamW", "RAdam"]),
        "batch_size": trial.suggest_int('batch_size', 4, 128),
        "num_workers": trial.suggest_int('num_workers', 1, 8),
        "shuffle": trial.suggest_categorical('shuffle', [True, False]),
        "train_transform": trial.suggest_categorical('train_transform', ["random_resize_crop", "random_horizontal_flip", "color_jitter", "none"]),
        "val_transform": trial.suggest_categorical('val_transform', ["resize", "center_crop", "none"]),
        "test_transform": trial.suggest_categorical('test_transform', ["resize", "center_crop", "none"])
    }
    
    # Get pretrained model
    model = get_model(params["model_name"])
    model = model.to(DEVICE)
    
    # Define criterion
    criterion = nn.CrossEntropyLoss()
    
    # Configure optimizer
    optimizer = getattr(
        torch.optim, params["optimizer_name"]
    )(model.parameters(), lr=params["lr"])
    
    # Create data loaders
    train_loader, val_loader, test_loader = create_data_loaders(batch_size=params["batch_size"], 
                                                                num_workers=params["num_workers"],
                                                                shuffle=params["shuffle"],
                                                                train_transform=params["train_transform"],
                                                                val_transform=params["val_transform"],
                                                                test_transform=params["test_transform"], 
                                                                train_df=train_df,
                                                                valid_df=valid_df,
                                                                train_images=train_images,
                                                                valid_images=valid_images)
    
    train_df, valid_df, train_images, valid_images
    # Train model
    best_model, best_acc = train_model(trial, model, criterion, optimizer, num_epochs=20)
    
    # Save best model for each trial
    # torch.save(best_model.state_dict(), f"model_trial_{trial.number}.pth")
    
    # Return accuracy (Objective Value) of the current trial
    return best_acc


In [ ]:
# sampler: We want to use a TPE sampler
# pruner: We use a MedianPruner in order to interrupt unpromising trials
# direction: The direction of study is “maximize” because we want to maximize the accuracy
# n_trials: Number of trials

sampler = optuna.samplers.TPESampler()    
study = optuna.create_study(
    sampler=sampler,
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=3, n_warmup_steps=5, interval_steps=3
    ),
    direction='maximize')
study.optimize(func=objective, n_trials=20)

In [ ]:
print("Best trial: ")
print(study.best_trial)